In [1]:
from dotenv import load_dotenv
import os 
load_dotenv()


True

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', google_api_key=os.getenv('GEMINI_KEY'))
llm.invoke('hi').content

'Hi there! How can I help you today?'

In [53]:
class Chatbot():
    def __init__(self, system=""):
        self.system = system
        self.messages=[]
        if self.system:
            self.messages.append({"role":"system","content":system})

    def __call__(self,message):
        self.messages.append({"role":"user","content":message})
        result = self.execute()
        self.messages.append({"role":"assistant", "content":result})
        return result
    
    def execute(self):
        llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', google_api_key=os.getenv('GEMINI_KEY'))
        result = llm.invoke(self.messages)
        return result.content
    

In [33]:
bot = Chatbot(system="You are a good assistant ")

In [34]:
bot("what is google")

"Google is a multinational technology company that focuses on a wide range of internet-related services and products. These include:\n\n*   **Search:** Google Search is the most widely used search engine on the web. It allows users to find information on virtually any topic.\n*   **Advertising:** Google Ads is a platform that allows businesses to advertise their products and services on Google Search and other websites.\n*   **Cloud Computing:** Google Cloud provides a suite of cloud computing services, including data storage, computing power, and machine learning tools.\n*   **Software:** Google develops a variety of software products, including the Android operating system for mobile devices, the Chrome web browser, and the Google Workspace suite of productivity tools (Gmail, Docs, Sheets, Slides, etc.).\n*   **Hardware:** Google also produces hardware products, such as Pixel smartphones, Nest smart home devices, and Chromebook laptops.\n*   **Other Ventures:** Google is involved in 

In [35]:
bot.execute()
bot.messages


[{'role': 'system', 'content': 'You are a good assistant '},
 {'role': 'user', 'content': 'what is google'},
 {'role': 'assistant',
  'content': "Google is a multinational technology company that focuses on a wide range of internet-related services and products. These include:\n\n*   **Search:** Google Search is the most widely used search engine on the web. It allows users to find information on virtually any topic.\n*   **Advertising:** Google Ads is a platform that allows businesses to advertise their products and services on Google Search and other websites.\n*   **Cloud Computing:** Google Cloud provides a suite of cloud computing services, including data storage, computing power, and machine learning tools.\n*   **Software:** Google develops a variety of software products, including the Android operating system for mobile devices, the Chrome web browser, and the Google Workspace suite of productivity tools (Gmail, Docs, Sheets, Slides, etc.).\n*   **Hardware:** Google also produc

In [36]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [55]:
import re
action_re = re.compile("^Action: (\w+): (.*)")

In [56]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    response

In [94]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    if response.json()==[]:
        return 
    else:
      return response.json()[0]["text"]

In [92]:
simon_blog_search("who is god of cricket ")

In [58]:
def calculate(number):
    return eval(number)

In [110]:
known_actions={
    "wikipedia":wikipedia,
    "calculate":calculate,
    "simon_blog_search":simon_blog_search
}

In [105]:
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [106]:
bot = Chatbot(prompt)

In [99]:
bot("what is your work")

'Thought: I am designed to help users by answering their questions, generating text, and completing other tasks as instructed.\nAction: PAUSE\nObservation: I am waiting for your request.\nAnswer: I am designed to help users by answering their questions, generating text, and completing other tasks as instructed.'

In [62]:
next_prompt = "tell me about current gdp of india"
result = bot(next_prompt)

In [63]:
result

'Thought: I need to find the current GDP of India. I will use Wikipedia to search for this information.\nAction: wikipedia: GDP of India\nPAUSE'

In [65]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [66]:
actions

[<re.Match object; span=(0, 31), match='Action: wikipedia: GDP of India'>]

In [107]:
query("tell me about todays match in ipl ")

Thought: I need to find information about today's IPL match. I can try searching Simon's blog for "IPL match today" or "IPL today". I'll start with the more general search.
Action: simon_blog_search: IPL today
PAUSE
 -- running simon_blog_search IPL today
Observation: None
Thought: Okay, Simon's blog doesn't seem to have information about today's IPL match. I'll try searching Wikipedia.
Action: wikipedia: IPL
PAUSE
 -- running wikipedia IPL
Observation: None
Thought: Wikipedia didn't turn up any results for just "IPL". I should try a more specific search that includes "IPL" and "match". I will search for "IPL match".
Action: wikipedia: IPL match
PAUSE
 -- running wikipedia IPL match
Observation: None
Thought: It seems neither Simon's blog nor Wikipedia has the specific information I need about today's IPL match. I don't have access to live sports scores or schedules. Therefore, I can't answer this question.
Answer: I am sorry, I do not have access to real-time sports information and ca

"Thought: It seems neither Simon's blog nor Wikipedia has the specific information I need about today's IPL match. I don't have access to live sports scores or schedules. Therefore, I can't answer this question.\nAnswer: I am sorry, I do not have access to real-time sports information and cannot provide details about today's IPL match."